In [1]:
!pip install openai
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00


In [17]:
from datasets import load_dataset

test_dataset = load_dataset('trec', split='test')
test_dataset = test_dataset.select(range(300))

train_dataset = load_dataset('trec', split='train')
#train_dataset = test_dataset.map(lambda example: {'label_name': "positive" if example["label"] == 1 else ("neutral" if example["label"] == 2 else "negative")})

train_dataset = test_dataset.map(lambda example: {'label_name': "entity" if example["coarse_label"] == 1 else \
                                                  "description" if example["coarse_label"] == 2 else \
                                                  "human being" if example["coarse_label"] == 3 else \
                                                  "location" if example["coarse_label"] == 4 else \
                                                  "numeric value"})


print(test_dataset)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'coarse_label', 'fine_label'],
    num_rows: 300
})


In [18]:
print(test_dataset[0])

{'text': 'How far is it from Denver to Aspen ?', 'coarse_label': 5, 'fine_label': 40}


In [19]:
import pandas as pd

df = pd.DataFrame(test_dataset)
df.head(10)

,text,coarse_label,fine_label
0,How far is it from Denver to Aspen ?,5,40
1,"What county is Modesto , California in ?",4,32
2,Who was Galileo ?,3,31
3,What is an atom ?,2,24
4,When did Hawaii become a state ?,5,39
5,How tall is the Sears Building ?,5,40
6,George Bush purchased a small interest in whic...,3,28
7,What is Australia 's national flower ?,1,14
8,Why does the moon turn orange ?,2,27
9,What is autism ?,2,24


In [20]:
df1 = pd.DataFrame(train_dataset)
df1.head(10)

,text,coarse_label,fine_label,label_name
0,How far is it from Denver to Aspen ?,5,40,numeric value
1,"What county is Modesto , California in ?",4,32,location
2,Who was Galileo ?,3,31,human being
3,What is an atom ?,2,24,description
4,When did Hawaii become a state ?,5,39,numeric value
5,How tall is the Sears Building ?,5,40,numeric value
6,George Bush purchased a small interest in whic...,3,28,human being
7,What is Australia 's national flower ?,1,14,entity
8,Why does the moon turn orange ?,2,27,description
9,What is autism ?,2,24,description


In [8]:
from google.colab import userdata
OPENAI_KEY = userdata.get('openai_api_key')

In [89]:
import os

from openai import OpenAI
client = OpenAI(api_key=OPENAI_KEY)

def sentiment_analysis(text):
  prompt = f"Classify the following question statement as either 'human being', 'location', 'numeric value', 'description' or 'entity'. \nSentence: {text}\nClass:"
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    temperature=0
  )
  return response.choices[0].message.content


In [90]:
import random,json
def few_shot_sentiment_analysis(text, k_shot=3):
  # add the system instructions - define the task
  messages = [
      {"role":"system","content":"Your task is to classify the following question statement as either 'human being', 'location', 'numeric value', 'description' or 'entity'."}
  ]

  # Add the few-shot examples using the `user` and `assistant`roles
  idxs = random.sample(list(range(len(train_dataset))), k_shot)
  for example in train_dataset.select(idxs):
    messages.append({
        "role": "user", "content": f"Sentence: {example['text']}"
    })
    messages.append({
        "role":"assistant", "content": f"Class: {example['label_name']}"
    })

  # add the target sentence as a user message
  messages.append({
      "role": "user", "content": f"Sentence: {text}"
  })

  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0
  )
  return response.choices[0].message.content.split(":")[-1].strip()


In [91]:

from datasets import load_metric
from tqdm import tqdm

metric = load_metric("accuracy")


In [92]:
import time
start = time.time()

#FEW SHOT

k_shot = 100 # @param
references = []
predictions = []
for item in tqdm(test_dataset):
  references.append(item['coarse_label'])
  pred = few_shot_sentiment_analysis(item['text'], k_shot)
  predictions.append(1 if pred == "entity" else \
                     2 if pred ==  "description" else \
                     3 if pred == "human being" else \
                     4 if pred == "location" else \
                     5)

metric.compute(predictions=predictions, references=references)

end = time.time()
print(end - start)

100%|██████████| 300/300 [09:06<00:00,  1.82s/it]

546.5513279438019


In [93]:
print(metric.compute(predictions=predictions, references=references)
)

{'accuracy': 0.9133333333333333}


In [99]:
text = "What city had a world fair in 1900 ?" # @param
class_label = sentiment_analysis(text)
print(f"The sentence '{text}' is classified as '{class_label}'.")

The sentence 'What city had a world fair in 1900 ?' is classified as 'location'.


In [98]:
text = "Why does the moon turn orange ?" # @param
class_label = sentiment_analysis(text)
print(f"The sentence '{text}' is classified as '{class_label}'.")

The sentence 'Why does the moon turn orange ?' is classified as 'description'.
